In [1]:
!pip install transformers trl

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [2]:
!pip install datasets

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [3]:
from transformers import AutoTokenizer, AutoModelForCausalLM

tokenizer = AutoTokenizer.from_pretrained("yankihue/gpt2-tr-uncontrolled-classification-news-economics-final")

model = AutoModelForCausalLM.from_pretrained("yankihue/gpt2-tr-uncontrolled-classification-news-economics-final")

tokenizer.eos_token_id = model.config.eos_token_id
tokenizer.pad_token = tokenizer.eos_token

Some weights of the model checkpoint at yankihue/gpt2-tr-uncontrolled-classification-news-economics-final were not used when initializing GPT2LMHeadModel: ['v_head.summary.bias', 'v_head.summary.weight']
- This IS expected if you are initializing GPT2LMHeadModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing GPT2LMHeadModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [4]:
from datasets import load_dataset

dataset = load_dataset("yankihue/turkish-news-categories",split="train")

In [5]:
import pandas as pd


In [6]:
dataset

Dataset({
    features: ['headline', 'category'],
    num_rows: 24097
})

In [7]:
def filter_fn(sample):
        category = sample["category"]
        return category == "ekonomi"  

dataset = dataset.filter(filter_fn, batched=False)

Filter:   0%|          | 0/24097 [00:00<?, ? examples/s]

In [8]:
df = dataset.to_pandas()

In [9]:
df

,headline,category
0,Borsa not artışı sonrası borsada sert yükseliş,ekonomi
1,"Trafik sorununa katlanabilir, elektrikli çözüm",ekonomi
2,Uçağa ayakta yolcu geliyor,ekonomi
3,FTden bakkal hesabı,ekonomi
4,Kadınlar çalışırsa tasarruflar artar,ekonomi
...,...,...
3260,Şoktaki emekliye taksit kolaylığı,ekonomi
3261,Avuç içinden kimlik doğrulama,ekonomi
3262,Vatandaş alacağı ev için 500 milyon fazladan ö...,ekonomi
3263,En şanssız iller,ekonomi


In [10]:
df_sampled = df.sample(n=3000)

In [11]:
df_sampled

,headline,category
403,Savunma sanayinde ihracat atağı,ekonomi
1454,11 AB üyesinden bankalara vergi,ekonomi
1005,Asya borsaları karışık seyretti,ekonomi
1247,"Yılda 1,5 milyar lira çöpe gidiyor",ekonomi
1349,Halı gibi parke taşı döşüyor,ekonomi
...,...,...
1895,Erdoğan Barajı'nın temeli atılıyor,ekonomi
854,Türkiye'nin yüzde 28'i devletin,ekonomi
53,Elektrik tüketiminde yeni dönem,ekonomi
208,"Twitter, hedefine Türkiye ile ulaşacak",ekonomi


In [12]:
import numpy as np

from trl.core import LengthSampler

In [13]:
def random_substring(s):
    if len(s) <= 10:
        return s
    elif 10 < len(s) < 40:
        end = np.random.randint(10, len(s))
        return s[:end]
    else:
        end = np.random.randint(10, 41)
        return s[:end]



In [14]:
df_sampled['headline_substring'] = df_sampled['headline'].apply(random_substring)

In [15]:
from transformers import pipeline

In [16]:
import torch

In [17]:
gen_kwargs = {"min_length": -1, "top_k": 0.0, "top_p": 1.0, "do_sample": True, "pad_token_id": tokenizer.eos_token_id}

In [18]:
# define an input text
input_text = "Gözaltındaki sendikacılar yar"

# encode the input text
input_ids = tokenizer.encode(input_text, return_tensors='pt')


output_min_length = 4
output_max_length = 16
output_length_sampler = LengthSampler(output_min_length, output_max_length)

gen_len=output_length_sampler()

# generate output from the model
output = model.generate(input_ids,max_new_tokens=gen_len, **gen_kwargs)

# decode the output to text
decoded_output = tokenizer.decode(output[0], skip_special_tokens=True)

print(decoded_output)

Gözaltındaki sendikacılar yaralandı Yol Gözlem Sanayi ve Emlak,


In [19]:
outputs = list()

In [20]:
for i in range(3000):
  input_text = df_sampled.iloc[i]['headline_substring']
  input_ids = tokenizer.encode(input_text, return_tensors='pt')
  output = model.generate(input_ids,max_new_tokens=gen_len, **gen_kwargs)
  decoded_output = tokenizer.decode(output[0], skip_special_tokens=True)
  outputs.append(decoded_output)

In [21]:
len(outputs)

3000

In [22]:
df2 = pd.DataFrame({"query": df_sampled['headline_substring'], "output": outputs },columns=["query","output"])

In [23]:
df2

,query,output
403,Savunma sanayin,"Savunma sanayin, UREKEC ve ISO 9001"
1454,11 AB üyesinden bankalara v,11 AB üyesinden bankalara v.b. satış ve sermay...
1005,Asya borsaları karışık,Asya borsaları karışık fiyatla getiri görüyor....
1247,"Yılda 1,5 milyar lira çöpe gidiyo","Yılda 1,5 milyar lira çöpe gidiyo. Ödemeler na..."
1349,Halı gibi parke taşı döşüy,Halı gibi parke taşı döşüy firmaların toptan s...
...,...,...
1895,Erdoğan Bar,"Erdoğan Baron, şehir 6 günde petrol fiyat"
854,Türkiye'nin yüzde 28'i,"Türkiye'nin yüzde 28'i, otomotiv sektöründe yü..."
53,Elektrik tük,"Elektrik tük çıkıyor, düşük fiyat ve Uludağ ih..."
208,"Twitter, hedefine Tü","Twitter, hedefine Tüomer Menkul Borsa ve Borsa..."


In [24]:
df2['output'] = df2.apply(lambda row: row['output'].replace(row['query'], ''), axis=1)


In [25]:
df2.to_csv("augmented_economy_news.csv",index = False)

In [26]:
df2

,query,output
403,Savunma sanayin,", UREKEC ve ISO 9001"
1454,11 AB üyesinden bankalara v,.b. satış ve sermaye yoluyla
1005,Asya borsaları karışık,fiyatla getiri görüyor. Borsa anals
1247,"Yılda 1,5 milyar lira çöpe gidiyo",. Ödemeler nakit motorlar Banka kredileri ve
1349,Halı gibi parke taşı döşüy,"firmaların toptan satış yolu, toptan satış"
...,...,...
1895,Erdoğan Bar,"on, şehir 6 günde petrol fiyat"
854,Türkiye'nin yüzde 28'i,", otomotiv sektöründe yükseldi.\n2010"
53,Elektrik tük,"çıkıyor, düşük fiyat ve Uludağ ihracatta"
208,"Twitter, hedefine Tü",omer Menkul Borsa ve Borsa firmalarıyla
